In [14]:
pip install tqdm requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
import requests
from tqdm.notebook import tqdm
from pathlib import Path

In [16]:
def create_folder(path):
    Path(path).mkdir(parents=True, exist_ok=True)

def download_image(url, path):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            with open(path, 'wb') as f:
                f.write(response.content)
            return True
    except Exception as e:
        pass
    return False

In [17]:
def get_pokemon_data(start: int, end: int):
    url = "https://pokeapi.co/api/v2/pokemon?limit=10000"
    response = requests.get(url)
    data = response.json()
    
    pokemon = []
    for entry in data['results']:
        name = entry['name']
        try:
            url_id = int(entry['url'].split('/')[-2])
        except:
            continue
        if start <= url_id <= end:
            pokemon.append((name, url_id))
    return pokemon


In [18]:
def check_image_exists(url: str) -> bool:
    #print(url)
    """Verificar se existe a imagem na URL fornecida"""
    try:
        response = requests.head(url, allow_redirects=True)
        return response.status_code == 200
    except requests.RequestException:
        return False

In [19]:
def baixar_pokemon(pokemon_name: str, pokemon_id: int, folder_path: str):
    forms = ["", "-mega", "-megax", "-megay", "-gmax", "-alola", "-galar", "-hisui", "-paldea"]
    
    # Formatar o nome da pasta como "XXXX - Nome do pokémon"
    folder_name = f"{str(pokemon_id).zfill(4)} - {pokemon_name.capitalize()}"
    folder = Path(folder_path) / folder_name
    folder.mkdir(parents=True, exist_ok=True)
    
    base_name = pokemon_name.lower().replace(".","").replace(" ","").replace("-","")
    base_id = str(pokemon_id).zfill(3)
    base_url = f"https://assets.pokemon.com/assets/cms2/img/pokedex/full/"
    
    for form in forms:
        form_name = base_name + form
        
        # PNGs (oficial e shiny)
        if form == "":
            # Oficial
            png_url = f"{base_url}{base_id}.png"
            #print(png_url)
            
            if not (check_image_exists(png_url)):
                continue
            
            download_image(png_url, folder / f"{form_name}.png")
            
            # Verificar formas em PNG
            suffix = 2
            while True:
                variation_url = f"{base_url}{base_id}_f{suffix}.png"
                success = download_image(variation_url, folder / f"{form_name}_f{suffix}.png")
                if not success:
                    break
                suffix += 1
        else:
            # Sprites alternativos que não existem no site oficial
            gif_url = f"https://play.pokemonshowdown.com/sprites/ani/{form_name}.gif"
            
            if not (check_image_exists(gif_url)):
                continue
                
            #shiny_sprite_url = f"https://play.pokemonshowdown.com/sprites/ani-shiny/{form_name}.gif"
            #download_image(gif_url, folder / f"{form_name}.png")
            #download_image(shiny_sprite_url, folder / f"{form_name}-shiny.png")
        
        # GIFs (normais e shiny)
        gif_url = f"https://play.pokemonshowdown.com/sprites/ani/{form_name}.gif"
        shiny_gif_url = f"https://play.pokemonshowdown.com/sprites/ani-shiny/{form_name}.gif"
        #print(shiny_gif_url)
        download_image(gif_url, folder / f"{form_name}.gif")
        download_image(shiny_gif_url, folder / f"{form_name}-shiny.gif")


In [20]:
# Caminho de destino
base_folder = r"C:\Users\corps\Documents\Cursor\github\Pokedex\jupyter\images"
#base_folder = r"C:\Users\artur\Documents\Pokedex\jupyter\images"
create_folder(base_folder)

# Faixa da Pokédex
start_id = int(input("Número inicial da Pokédex: "))
end_id = int(input("Número final da Pokédex: "))

# Obter lista
lista_pokemon = get_pokemon_data(start_id, end_id)

# Baixar todos
for name, pid in tqdm(lista_pokemon):
    baixar_pokemon(name, pid, base_folder)


Exception ignored in: <function tqdm.__del__ at 0x0000020B3172BF60>
Traceback (most recent call last):
  File "C:\Users\artur\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "C:\Users\artur\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html